In [1]:
import pandas as pd

# Data Loading
Merge all the reports into one.

In [2]:
# computer:
import socket
hostname = socket.gethostname()

if hostname == 'arch':
    directory = '/home/sergio/Documents/TELMEX/Productividad/Datos/2020/rda/'
else:
    directory = '/home/sergio/Documents/Telmex/Productividad/Data/2020/rda/'


file_prefix = 'rdat_metro'

df = pd.DataFrame()

import os
rda_files = []
columns = []
for f in os.listdir(directory):
    if f.startswith(file_prefix):
        df_tmp = pd.read_excel(directory + f, sheet_name='base')
        #print(f, '-', df_tmp.shape)
        columns.append(df_tmp.columns)
        df = df.append(df_tmp)
        
#print(df.shape)

# ==============
# Split data by types
# ==============
date_cols = df.select_dtypes('datetime')
int_cols = df.select_dtypes('number')
obj_cols = df.select_dtypes(object)

# Common errors

---
#### Hipótesis 1
Las fallas se repiten. Si se repiten podemos pensar que una solución ayudará a la mayoría de las fallas.

#### Prueba de hipótesis
Revisar qué tanto se repiten las fallas.

In [33]:
obj_cols[obj_cols['CMANTENI'] == 'CMABS']['FALLAENC'].value_counts()[:8]

FALLA EQUIPO DE TRANSPORTE AEP                                                 2
Da¤o en modulo del cor-director en Ctl Estrella, se migraron a otro puerto.    2
FALLA EQUIPO DE TRANSPORTE AEP.                                                2
RESET DE PUERTO EN CENTRAL ESTRELLA.                                           1
LIMPIEZA   DE   F.  OPTICA                                                     1
Falla en trk                                                                   1
ALARMA DE [RUP] EN EL NODO CDMXDFESHAG ...r01s1b28p01                          1
Cambio de puerto en CTL ES (del puerto 9 al puerto 3).                         1
Name: FALLAENC, dtype: int64

#### Observaciones
Las fallas son reportadas por texto no códigos entonces es difícil encontrar repeticiones (quien reporta la falla la reporta como se le ocurre) aunque puedan estar relacionadas (por ejemplo, las fallas relacionadas a mantenimiento están relacionadas entre sí).

---
#### Hipótesis 2
Encontrar datos repetidos en las columnas intentando encontrar patrones.

#### Prueba de hipótesis
Enfocarse en un CM (Abastos) y revisar el total de fallas. Después revisar cuántos datos se repiten por columna y ver si esa cantidad es significativa con respecto al total de fallas.

Total de fallas:

In [42]:
total_fallas = obj_cols[obj_cols['CMANTENI'] == 'CMABS'].shape[0]
print(f'{total_fallas = }')

total_fallas = 59


#### Revisar los datos más repetidos por cada columna
A continuación se imprimen los 8 datos más repetidos por columna, junto con su porcentaje respecto al total de fallas.

In [45]:
for col in obj_cols.columns:
    values = pd.DataFrame(obj_cols[obj_cols['CMANTENI'] == 'CMABS'][col].value_counts()[:8])
    values['percentage'] = values / 59 * 100
    print(values)
    print()
    
    

               ID  percentage
A32-1110-0138   2    3.389831
A04-1710-0329   1    1.694915
D32-1306-0368   1    1.694915
A32-1510-0870   1    1.694915
D32-1606-0436   1    1.694915
D03-0512-0010   1    1.694915
C05-1809-0114   1    1.694915
A32-1511-0299   1    1.694915

                 SITIO  percentage
MEXOET              46   77.966102
ES_ MEXICO D.F.     12   20.338983
AB_ MEXICO D.F.      1    1.694915

      SERVICIO  percentage
E1          25   42.372881
AXX         12   20.338983
DS0          8   13.559322
CETH         5    8.474576
ETRN         3    5.084746
E0           3    5.084746
TKD          2    3.389831
ST1          1    1.694915

     CENTRAL  percentage
ES_       49   83.050847
AB_        8   13.559322
MY_        1    1.694915
IK_        1    1.694915

        CATENCIO  percentage
CASCAF        27   45.762712
CASMEX        20   33.898305
CAO           10   16.949153
CARN           1    1.694915
CASQRO         1    1.694915

       CMANTENI  percentage
CMABS        59